In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
import pdb
import gym
import copy
import functools
import matplotlib.pyplot as plt

import torch
import numpy as np
from collections import deque

from Config import Config
from Models import ActorCritic
from PPO import PPOPixel
from Networks import cnn_head_model, actor_model, critic_model, head_model
from Memory import Memory
# from baselines.common.cmd_util import make_env
from baselines.common.atari_wrappers import wrap_deepmind, make_atari


env_id = "BreakoutNoFrameskip-v4"
env = make_atari(env_id)
env = wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=True, scale=False)

config = Config(env)

config.update_every = 500
config.num_learn = 4
config.win_condition = 230
config.n_episodes = 2000000
config.max_t = 700
config.lr = 2.5e-4
config.hidden_size = 512

config.Memory = Memory
config.Model = ActorCritic
config.head_model = functools.partial(cnn_head_model, config)
config.actor_model = functools.partial(actor_model, config)
config.critic_model = functools.partial(critic_model, config)


Running experiment with device: cuda:0


In [ ]:
def train(config):
    env = copy.deepcopy(config.env)
    steps = 0
    scores_deque = deque(maxlen=100)
    scores = []
    average_scores = []
    max_score = -np.Inf

    agent = PPOPixel(config)

    for i_episode in range(1, config.n_episodes+1):
        state = env.reset()
        score = 0
        while True:
            steps += 1

            action, log_prob, value, entr = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            agent.add_to_mem(state, action, reward, log_prob, done)

            # Update 
            state = next_state
            score += reward

            if steps >= config.update_every:
                agent.learn(config.num_learn)
                agent.mem.clear()
                steps = 0

            if done:
                break 

        # Book Keeping
        scores_deque.append(score)
        scores.append(score)
        average_scores.append(np.mean(scores_deque))

        if i_episode % 10 == 0:
            print("\rEpisode {}	Average Score: {:.2f}	Score: {:.2f}".format(i_episode, np.mean(scores_deque), score), end="")
        if i_episode % 1000 == 0:
            print("\rEpisode {}	Average Score: {:.2f}".format(i_episode, np.mean(scores_deque)))   

        if np.mean(scores_deque) > config.win_condition:
            print("\nEnvironment Solved!")
            break

    return scores, average_scores

scores, average_scores = train(config)

Episode 700	Average Score: 0.31	Score: 0.00